<a href="https://colab.research.google.com/github/saurabhgangw/Electron-Spin-Resonance-Experiment-/blob/master/CTAModa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Author: Saurabh Gangwar
# Deep Learning Model to classify CTA events

# In[ ]:
#import modules

import tables
import numpy as np
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, BatchNormalization, Dropout, TimeDistributed,LSTM
from tensorflow.keras.layers import ReLU, MaxPool2D, AvgPool2D, GlobalAvgPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras_tuner as kt
import time
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
# In[ ]:


# Filenames of Data used for training, validation and testing.
# No duplicates, so testing and validation are independant of training
train_split, val_split, test_split = 0.7,0.15,0.15

all_filenames = [1028155,    15760266,   21217180,   26227461, 47142250,
10925472,   158272236,  2126965,    263287239,  48226249,
110182113,  15929268,   21317083,   26411052,   4920265,
111177194,  160200191,  214847,     26524833,   50197128,
11252272,   162187274,  216259223,  267199235,  51224255,
113264112,  16325284,   217167200,  268150297,  512938,
114217205,  164277231,  21822021,   26953213,   527069,
115238172,  1649133,    2183160,    270260186,  5518114,
11680281,   165155260,  219207204,  2711201,    56243224,
1172193,    16636157,   2207142,    2715567,    5721995,
1175462,    16781288,   22168182,   272131233,  58276161,
118164251,  1689582,    2224370,    273288152,  5919277,
1194101,    169225189,  2229897,    27529287,   6092270,
1198299,    170293184,  2233824,    276186179,  61190258,
120111131,  17112323,   22410294,   27783174,   6144259,
12124183,   17219311,   2258675,    27884169,   6226554,
12189283,   17388146,   2265620,    27996195,   63262134,
122236173,  17424071,   22739228,   280247206,  64297273,
123252167,  175230129,  22813486,   281283175,  659188,
125241227,  178148246,  2297798,    282273243,  6623344,
126116211,  179253150,  23012589,   283138105,  67159298,
12731132,   180146263,  231188190,  284166285,  6816,
128179229,  181174149,  2329241,    2851192,    6929026,
129175280,  182208135,  233015,     285189279,  7027085,
130114118,  183169144,  233215156,  286117106,  71156247,
131172185,  183737,     234239257,  28879125,   7217271,
13227884,   18414366,   2354831,    28932196,   7310991,
13275120,   185296220,  237295147,  29076110,   7465278,
1332237,    186184207,  23862140,   2911539,    7515851,
13421474,   187250165,  2394100,    293263264,  761520,
135280254,  18864103,   2406394,    2942203,    77350,
136122230,  189242,     24128257,   294244291,  7810587,
137162261,  19020648,   242237158,  29721166,   7924276,
138154218,  19107245,   2423275,    298267139,  8093176,
139176124,  191140177,  2431556,    299286214,  8212712,
14021241,   192232122,  24457143,   30271138,   8268111,
141168290,  193294238,  24590248,   323173,     8312234,
14119219,   19425536,   246120151,  323440,     851976,
14240153,   1952918,    247289268,  33139299,   8687293,
14319160,   19622226,   24822278,   34221240,   875292,
144165232,  198113108,  249115221,  3523564,    88257180,
145151168,  199284212,  25094225,   3697109,    8985202,
146108141,  200251215,  251118296,  37112267,   9044102,
147203295,  20145130,   252205199,  38995,      9178162,
1486646,    203195171,  2526222,    39126115,   9227178,
14913244,   20413093,   25524679,   40204289,   94285256,
150133253,  20558117,   256201269,  416282,     96173210,
15171159,   206132181,  257147137,  42137170,   9716334,
1526190,    20715796,   258216262,  4279242,    974126,
1537527,    2085035,    2594722,    43229198,   9833127,
154213123,  20935163,   26016181,   44121277,   99196136,
1545216,    21025892,   261227119,  45281252]

filenames_train = []
filenames_val = []
filenames_test = []
for i in range(len(all_filenames)):
    if i/len(all_filenames) < train_split:
        filenames_train.append(str(all_filenames[i]))
    elif i/len(all_filenames) > 1 - test_split:
        filenames_test.append(str(all_filenames[i]))
    elif i/len(all_filenames) > 1 - val_split - test_split:
        filenames_val.append(str(all_filenames[i]))

print(filenames_test)
print(filenames_val)
print(len(filenames_train), "files will be used for training")
print(len(filenames_val), "files will be used for validating")
print(len(filenames_test), "files will be used for testing")


# In[ ]:


# Load data from files and for monoscopic data: clean
def open_data_images(file_path, model_type):
    # Loading data
    current_file = tables.open_file(file_path,'r')
    # Masking out electron events and reading peak timing data with corresponding labels
    mask_electrons = current_file.root.event_label[:] != 2
    peak_times = current_file.root.peak_times[:,:,:,:][mask_electrons]
    event_labels = current_file.root.event_label[:][mask_electrons]
    # Add extra dimension for channels
    peak_times = peak_times.reshape((*np.shape(peak_times),1))
    event_labels = event_labels.reshape((np.shape(event_labels)[0],1))

    if model_type == "mono":
        data_images = np.empty((0, 48,48,1))
        data_labels = np.empty((0, 1))
        # Add images from all 4 telescopes to one array
        for i in range(0,4):
            data_images = np.concatenate((data_images,peak_times[:,i,:,:]), axis = 0)
            data_labels = np.concatenate((data_labels,event_labels[:]))
        # Mask out images where all data is 0 for monoscopic training
        zero_mask = np.sum(data_images, axis = 1) # sums over axis = 1,so from shape (N,48,48,1) to (N,48,1) (along one axis of the image)
        zero_mask = np.sum(zero_mask, axis = 1) # sums over axis = 1,so from shape (N,48,1) to (N,1) (along other axis of the image)
        zero_mask = np.squeeze(zero_mask) # goes from shape (N,1) to shape (N,)
        zero_mask = zero_mask > 100 # True/False where sum of whole image is  > 0
        data_images = data_images[zero_mask,:,:,:]
        data_labels = data_labels[zero_mask]

    if model_type == "stereo":
        data_images = np.empty((0, 4, 48,48,1))
        data_labels = np.empty((0, 1))
        data_labels = np.concatenate((data_labels,event_labels[:]))
        data_images = np.concatenate((data_images,peak_times), axis = 0)

    return data_images, data_labels


# In[ ]:


# loads batches of data
class DataGenerator(keras.utils.Sequence):
    def __init__(self, filenames, batch_size=64, shuffle=True, model_type = "mono"):
        # Initialization
        self.batch_size = batch_size # Batch Size
        self.shuffle = shuffle # Shuffle the data files read and the batches within the files
        self.model_type = model_type # stereo or mono
        self.filenames = np.array(filenames) # Names of data files
        self.file_sizes = np.array([]) # Amount of useable sets of images (if mono -> filtered) per file
        self.next_file_no = 0 # Counter of which file is read next
        self.current_batch = 0 # Counter of which is the current loaded batch

        # Loop over all files to know how many batches are in each file (and therefore how many batches in total)
        for filename in self.filenames:
            # Update this path to your dataset directory
            file_path = os.path.join(r'/path/to/your/dataset', filename + '.hdf5')
            data_images, data_labels  = open_data_images(file_path, self.model_type)
            self.file_sizes = np.append(self.file_sizes, np.shape(data_images)[0])
            self.batches_per_file = np.floor(self.file_sizes/self.batch_size) # Amount of batches in each file

        self.reset_counters() # Resets batch and file counters and shuffles them

    def __len__(self): # Defines the Function len(DG), where DG is an object of class DataGenerator.
        #len(DG) is called by model.fit and keras tuner to know how many batches are processed (example: the 1/527 counter)
        return int(np.floor(np.sum(self.batches_per_file)))

    def __getitem__(self, index): # Defines the Function DG[i], where DG is an object of class DataGenerator and i is an index.
        #DG[i] is called by model.fit and keras tuner to open batch i/len(DG). In this function, the index i will
        # be discarded. Every time an element is accessed via DG[x] the next batch of a file is loaded (maybe shuffled).
        # If it would be batch 0, the file is loaded, and if it would be the last batch, the batch counter is reset to 0
        if self.current_batch == 0: #Check if the current batch to load is the first batch

            self.current_batch_mixed = np.arange(0, self.batches_per_file[self.next_file_no]+1) # Define order
                # in which to open the batches of the file
            if self.shuffle == True: # permutate the order in which the batches are opened in the file
                p = np.random.permutation(len(self.current_batch_mixed)) # create permuation
                self.current_batch_mixed = self.current_batch_mixed[p] #apply permutation
            filename = self.filenames[self.next_file_no] # Get filename to open
            self.next_file_no += 1 # Prepare next filename to open
            # Update this path to your dataset directory
            file_path = os.path.join(r'/path/to/your/dataset', filename + '.hdf5')
            self.data_images, self.data_labels = open_data_images(file_path, self.model_type)

        # Input (image) Data X of shape (batchsize, *dims)
        # self.current_batch_mixed[self.current_batch] calls correct (shuffled) batch
        # Example first batch (self.current_batch=0) which happens to shuffled to third position self.current_batch_mixed[0]=3
        if self.model_type == "mono":
            X = self.data_images[int(self.current_batch_mixed[self.current_batch]*self.batch_size):
                                          int((self.current_batch_mixed[self.current_batch]+1)*self.batch_size),:,:,:]
        if self.model_type == "stereo":
            X = self.data_images[int(self.current_batch_mixed[self.current_batch]*self.batch_size):
                                          int((self.current_batch_mixed[self.current_batch]+1)*self.batch_size),:,:,:,:]

        # Output (label) data y of shape (batchsize, 1)
        y = self.data_labels[int(self.current_batch_mixed[self.current_batch]*self.batch_size):
                                      int((self.current_batch_mixed[self.current_batch]+1)*self.batch_size)]
         # Convert y to shape (batchsize, 2), binary data
        y = keras.utils.to_categorical(y, num_classes=2)

        self.current_batch +=1 # End of batch loading, increase counter
        # If it was last batch of file, reset batch counter to 0 so new file is loaded
        if self.current_batch > self.batches_per_file[self.next_file_no-1]:
            self.current_batch = 0
        return X, y

    def reset_counters(self): #Reset Batch and file counter
        self.next_file_no = 0
        self.current_batch = 0
        # Shuffle the files
        if self.shuffle == True:
            p = np.random.permutation(len(self.filenames)) # create permuation
            self.filenames = self.filenames[p]  # apply permutation to file names
            self.file_sizes = self.file_sizes[p] # apply same permutation to file sizes
            self.batches_per_file = self.batches_per_file[p] # apply same permutation to batches per file



# In[ ]:

# Method called at the beginning of each epoch to reset the batch counters of the data generators
class OnEpochBegin(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        mono_training_generator.reset_counters()
        mono_validation_generator.reset_counters()
        stereo_training_generator.reset_counters()
        stereo_validation_generator.reset_counters()
        print("Epoch Begin")



# In[ ]:


# Define Function to build and optimize a net for monoscopic image analysis
def build_mono_net(hp):
    inputs = Input(shape = (48, 48, 1)) #input layer
    net = inputs

    # Optimize amount of convolutional blocks
    for i in range(hp.Int('conv_blocks',min_value = 3, max_value = 5, default=3)):
        # Optimize amount of neurons per convolutional blocks
        filters = hp.Int('filters_' + str(i), min_value = 32,max_value = 256, step=8)
        conv_dropout_rate = hp.Float('conv_droput_rate_'+ str(i), min_value = 0.05, max_value = 0.5, step=0.05)

        # Define the Dropout, conv, BatchNorm and activation layers for each block
        net = Dropout(conv_dropout_rate)(net)
        net = Convolution2D(filters, kernel_size=(3, 3), padding= 'same')(net)
        #net = BatchNormalization()(net)
        net = ReLU()(net)

    # Optimize for MaxPooling or Average Pooling
        if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
            net = MaxPool2D()(net)
        else:
            net = AvgPool2D()(net)

    # Core net as global, so it can be saved after training
    global core_net
    core_net = Model(inputs,net)

    net = Flatten()(net)

     # Optimize amount of dense layers
    for i in range(hp.Int('dense_layers',min_value = 1, max_value = 3, default=1)):
        dense_dropout_rate = hp.Float('dense_droput_rate_'+ str(i), min_value = 0.05, max_value = 0.5, step=0.05)
        dense_neurons = hp.Int('dense_neurons_' + str(i), min_value = 10, max_value = 200, step = 10, default = 50)
        net = Dropout(dense_dropout_rate)(net)
        net = Dense(dense_neurons, activation = "relu")(net)

    # Output Layer
    dense_dropout_rate = hp.Float('dense_droput_rate_-1', min_value = 0.05, max_value = 0.5, step=0.05)
    net = Dropout(dense_dropout_rate)(net)
    outputs = Dense(2, activation= 'sigmoid')(net)

    # Create Model
    learning_rate = hp.Float('learning_rate', min_value = 1e-4, max_value =1e-1, sampling='log')
    mono_model = Model(inputs, outputs)
    mono_model.compile(optimizer= Adam(learning_rate),
                loss= 'BinaryCrossentropy', metrics = ['accuracy'])
    return mono_model


# In[ ]:


core_net = None # Create Variable
#initiate generators (as global, so their counters can be reset)
global mono_training_generator
global mono_validation_generator
global mono_test_generator
global stereo_training_generator
global stereo_validation_generator
global stereo_test_generator
mono_training_generator = DataGenerator(filenames_train, batch_size=64, shuffle=True, model_type = "mono")
mono_validation_generator = DataGenerator(filenames_val, batch_size=64, shuffle=True, model_type = "mono")
mono_test_generator = DataGenerator(filenames_test, batch_size=64, shuffle=False, model_type = "mono")
stereo_training_generator = DataGenerator(filenames_train, batch_size=64, shuffle=True, model_type = "stereo")
stereo_validation_generator = DataGenerator(filenames_val, batch_size=64, shuffle=True, model_type = "stereo")
stereo_test_generator = DataGenerator(filenames_test, batch_size=64, shuffle=False, model_type = "stereo")


# In[ ]:


# Initialize tuner to run the mono_model using the Hyperband search algorithm
tuner = kt.Hyperband(
    hypermodel = build_mono_net,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2,
    directory="MoDAII_Optimization",
    project_name="Mono_Model")


# In[ ]:


callbacks = [tf.keras.callbacks.EarlyStopping(patience=3),
             OnEpochBegin()]

tuner.search(mono_training_generator,
             validation_data= mono_validation_generator,
             epochs=30,
             callbacks=callbacks)

# In[ ]:

# Get the optimal hyperparameters
best_mono_hps= tuner.get_best_hyperparameters(1)[0]

# Get the best model
best_mono_model = tuner.get_best_models(1)[0]

# Show model summary
best_mono_model.summary()

with open(r"/home/hpc/b129dc/b129dc22/LargeDataSet/LargeDataSet_Summary_Monomodel.txt", 'w') as f:
    best_mono_model.summary(print_fn=lambda x: f.write(x + '\n'))

# In[ ]:


# Integrate early stopping
early_stopping =EarlyStopping(monitor='val_loss', patience=20)
bst_mono_net_path = r'/home/hpc/b129dc/b129dc22/LargeDataSet/bst_mono_net_path.h5'
mono_net_checkpoint = ModelCheckpoint(bst_mono_net_path, save_best_only=True, save_weights_only=True)

# Run Learning
mono_net = tuner.hypermodel.build(best_mono_hps)
history = mono_net.fit(mono_training_generator,
          validation_data=mono_validation_generator,
          epochs= 100,
         callbacks=[early_stopping, mono_net_checkpoint, OnEpochBegin()])

np.save(r'/home/hpc/b129dc/b129dc22/LargeDataSet/history_mono.npy',history.history)

# Get best epoch and save mono and core nets (set all params non trainable, to avoid errors)
for l in mono_net.layers:
    l.trainable = False
mono_net.load_weights(bst_mono_net_path)
mono_net.save("mono_net")
core_net.save('core_net')

# Evaluate Result
eval_result = mono_net.evaluate(mono_test_generator)
print(f"test loss: {eval_result[0]}, test accuracy: {eval_result[1]}")

text_file = open("/home/hpc/b129dc/b129dc22/LargeDataSet/LargeDataSet_Testresults_Monomodel.txt", "w")
text_file.write(f"test loss: {eval_result[0]}, test accuracy: {eval_result[1]}")
text_file.close()

# In[ ]:


# Define Function to build and optimize a net for stereoscopic image analysis
def build_stereo_net(hp):
    core_net = keras.models.load_model('core_net') # Load core net from file
    inputs = Input(shape = (4,48, 48, 1)) #input layer
    net = inputs

    net = TimeDistributed(core_net, trainable=False)(net) # Apply core net to all 4 telescoptes
    net = TimeDistributed(Flatten())(net) # Necessary to input to LSTM

    #Optimize amount of lstm blocks
    for i in range(hp.Int('lstm_blocks',min_value = 0, max_value = 5, default=0)):
        dropout_rate = hp.Float('lstm_droput_rate_'+str(i), min_value = 0.05, max_value = 0.5, step=0.05)
        net = Dropout(dropout_rate)(net)
        net = LSTM(4,return_sequences=True)(net)

    dropout_rate = hp.Float('lstm_droput_rate_-1', min_value = 0.05, max_value = 0.5, step=0.05)
    net = Dropout(dropout_rate)(net)
    net = LSTM(4,return_sequences=False)(net)

     # Optimize amount of dense layers
    for i in range(hp.Int('dense_layers',min_value = 1, max_value = 5, default=2)):
        dense_dropout_rate = hp.Float('dense_droput_rate_'+ str(i), min_value = 0.05, max_value = 0.5, step=0.05)
        dense_neurons = hp.Int('dense_neurons_' + str(i), min_value = 10, max_value = 200, step = 10, default = 50)
        net = Dropout(dense_dropout_rate)(net)
        net = Dense(dense_neurons, activation = "relu")(net)

    # Output Layer
    dropout_rate = hp.Float('dense_droput_rate_-1', min_value = 0.05, max_value = 0.5, step=0.05)
    net = Dropout(dropout_rate)(net)
    outputs = Dense(2, activation= 'sigmoid')(net)

    # Create Model
    learning_rate = hp.Float('learning_rate', min_value = 1e-4, max_value =1e-1, sampling='log')
    stereo_model = Model(inputs, outputs)
    stereo_model.compile(optimizer= Adam(learning_rate),
                loss= 'BinaryCrossentropy', metrics = ['accuracy'])
    return stereo_model


# In[ ]:

# Initialize tuner to run the stereo_model using the Hyperband search algorithm
tuner = kt.Hyperband(
    hypermodel = build_stereo_net,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2,
    overwrite=False,
    directory="MoDAII_Optimization",
    project_name="Stereo_Model")


# In[ ]:


# Run the search for best stereo_model
callbacks = [tf.keras.callbacks.EarlyStopping(patience=3),
             OnEpochBegin()]

tuner.search(stereo_training_generator,
             validation_data= stereo_validation_generator,
             epochs=30,
             callbacks=callbacks)


# In[ ]:


# Get the optimal hyperparameters
best_stereo_hps= tuner.get_best_hyperparameters(1)[0]

# Get the best model
best_stereo_model = tuner.get_best_models(1)[0]

# Show model summary
best_stereo_model.summary()

with open(r"/home/hpc/b129dc/b129dc22/LargeDataSet/LargeDataSet_Summary_Stereomodel.txt", 'w') as f:
    best_stereo_model.summary(print_fn=lambda x: f.write(x + '\n'))

# In[ ]:


# Build the stereo_net with the optimal hyperparameters and train the model.
# Integrate early stopping
early_stopping =EarlyStopping(monitor='val_loss', patience=10)
bst_stereo_net_path = r'/home/hpc/b129dc/b129dc22/LargeDataSet/bst_stereo_net_path.h5'
stereo_net_checkpoint = ModelCheckpoint(bst_stereo_net_path, save_best_only=True, save_weights_only=True)

# Run Learning
stereo_net = tuner.hypermodel.build(best_stereo_hps)
history = stereo_net.fit(stereo_training_generator,
          validation_data= stereo_validation_generator,
          epochs= 100, batch_size = 64,
         callbacks=[early_stopping, stereo_net_checkpoint, OnEpochBegin()])

np.save(r'/home/hpc/b129dc/b129dc22/LargeDataSet/history_stereo.npy',history.history)

# Get best epoch and save stereo net (set all params non trainable, to avoid errors)
for l in stereo_net.layers:
    l.trainable = False
stereo_net.load_weights(bst_stereo_net_path)
stereo_net.save("stereo_net")

# Evaluate the result
eval_result = stereo_net.evaluate(stereo_test_generator)
print(f"test loss: {eval_result[0]}, test accuracy: {eval_result[1]}")

text_file = open("/home/hpc/b129dc/b129dc22/LargeDataSet/LargeDataSet_Testresults_Stereomodel.txt", "w")
text_file.write(f"test loss: {eval_result[0]}, test accuracy: {eval_result[1]}")
text_file.close()

# In[ ]:
########################################### From here: Plotting

#plotting the history of a net training
def history_analysis(history, name="Test"):
    fig, ax = plt.subplots(1,2, figsize = (12,4))
    #plot accuracy
    #plt.rcParams.update({'font.size': fontssize})
    ax[0].plot(history['accuracy'], label='Accuracy',color='red')
    ax[0].plot(history['val_accuracy'], label = 'Validation accuracy',color='blue')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Accuracy')
    ax[0].set_ylim([0.5, 1])
    ax[0].legend(loc='best')
    ax[0].grid(alpha=0.6)
    #plot loss
    ax[1].plot(history['loss'],color='red',ls='dashed')
    ax[1].plot(history['val_loss'],color='blue',ls='dashed')
    ax[1].set_ylabel('Loss')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Loss', 'Validation Loss'], loc='best')
    ax[1].grid(alpha=0.6)
    file_path = os.path.join(r"/home/hpc/b129dc/b129dc22/Analysis", name + '_history.png')
    plt.savefig(file_path)
    plt.show()

    #plot the ROC curve of a model
def plot_roc_curve(net, X_truth, y_truth, name = "Large Data Set", batch_size = 64):
    y_pred_keras = net.predict(X_truth, batch_size=batch_size)[:,1] #Column 0 is prob. of being 0 and Column 2 is prob of being 1
    #eval_result = net.evaluate(test_generator)
    #print(f"test loss: {eval_result[0]}, test accuracy: {eval_result[1]}")
    fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_truth, y_pred_keras)
    auc_keras = auc(fpr_keras, tpr_keras)
    plt.plot(fpr_keras, tpr_keras, label=name +' (area = {:.3f})'.format(auc_keras))


# In[ ]:

# Plot history of Small Mono Model

history=np.load(r"/home/hpc/b129dc/b129dc22/SmallDataSet/history_mono.npy",
                allow_pickle='TRUE').item()
#mono_test_prediction = mono_net.predict(mono_test_generator)
history_analysis(history, name = "mono_net_small")


# In[ ]:
# Plot history of Mid Mono Model

history=np.load(r"/home/hpc/b129dc/b129dc22/MidDataSet/history_mono.npy",
                allow_pickle='TRUE').item()
#mono_test_prediction = mono_net.predict(mono_test_generator)
history_analysis(history, name = "mono_net_mid")


# In[ ]:
# Plot history of Large Mono Model

history=np.load(r"/home/hpc/b129dc/b129dc22/LargeDataSet/history_mono.npy",
                allow_pickle='TRUE').item()
#mono_test_prediction = mono_net.predict(mono_test_generator)
history_analysis(history,name = "mono_net_large")


# In[ ]:

# Load X_truth and y_truth to variables

mono_test_generator.reset_counters()
y_truth = np.empty((0,2))
X_truth = np.empty((0,48,48,1))
for i in range(len(mono_test_generator)):
    next_batch_X, next_batch_y = mono_test_generator[i]
    y_truth = np.concatenate((y_truth, next_batch_y))
    X_truth = np.concatenate((X_truth, next_batch_X))
y_truth = y_truth[:,1]

print("Loading Data into Variables DONE")


# Plot ROC curve of Mono Model

plt.figure(figsize = (5,5), dpi = 600)
mono_net = tf.keras.models.load_model(r"/home/hpc/b129dc/b129dc22/SmallDataSet/mono_net")
plot_roc_curve(mono_net,  X_truth, y_truth,  name = "Small Data Set")
print("ROC 1 DONE")
mono_net = tf.keras.models.load_model(r"/home/hpc/b129dc/b129dc22/MidDataSet/mono_net")
plot_roc_curve(mono_net,  X_truth, y_truth,  name = "Mid Data Set")
print("ROC 2 DONE")
mono_net = tf.keras.models.load_model(r"/home/hpc/b129dc/b129dc22/LargeDataSet/mono_net")
plot_roc_curve(mono_net,  X_truth, y_truth,  name = "Large Data Set")
print("ROC 3 DONE")



plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.grid(alpha=0.6)
plt.xlim(0,1)
plt.ylim(0,1)
plt.legend()
file_path = (r"/home/hpc/b129dc/b129dc22/Analysis/mono_net_roc.png")
plt.savefig(file_path)
plt.show()

# In[ ]:

# Plot history of Small Stereo Model

history=np.load(r"/home/hpc/b129dc/b129dc22/SmallDataSet/history_stereo.npy",
                allow_pickle='TRUE').item()
#mono_test_prediction = mono_net.predict(mono_test_generator)

history_analysis(history,name = "stereo_net_small")


# In[ ]:

    # Plot history of Mid Stereo Model

history=np.load(r"/home/hpc/b129dc/b129dc22/MidDataSet/history_stereo.npy",
                allow_pickle='TRUE').item()
#mono_test_prediction = mono_net.predict(mono_test_generator)

history_analysis(history, name = "stereo_net_mid")


# In[ ]:

# Plot history of Large Stereo Model

history=np.load(r"/home/hpc/b129dc/b129dc22/LargeDataSet/history_stereo.npy",
                allow_pickle='TRUE').item()
#mono_test_prediction = mono_net.predict(mono_test_generator)
history_analysis(history, name = "stereo_net_large")


# In[ ]:

# Load X_truth and y_truth to variables

stereo_test_generator.reset_counters()
y_truth = np.empty((0,2))
X_truth = np.empty((0,4,48,48,1))
for i in range(len(stereo_test_generator)):
    next_batch_X, next_batch_y = stereo_test_generator[i]
    y_truth = np.concatenate((y_truth, next_batch_y))
    X_truth = np.concatenate((X_truth, next_batch_X))
y_truth = y_truth[:,1]
print("Loading Data into Variables DONE")

# Plot ROC curve of Stereo Model

plt.figure(figsize = (5,5), dpi = 600)
stereo_net = tf.keras.models.load_model(r"/home/hpc/b129dc/b129dc22/SmallDataSet/stereo_net")
plot_roc_curve(stereo_net, X_truth, y_truth, name = "Small Data Set")
print("ROC 1 DONE")
stereo_net = tf.keras.models.load_model(r"/home/hpc/b129dc/b129dc22/MidDataSet/stereo_net")
plot_roc_curve(stereo_net, X_truth, y_truth, name = "Mid Data Set")
print("ROC 2 DONE")
stereo_net = tf.keras.models.load_model(r"/home/hpc/b129dc/b129dc22/LargeDataSet/stereo_net")
plot_roc_curve(stereo_net, X_truth, y_truth, name = "Large Data Set")
print("ROC 3 DONE")

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.grid(alpha=0.6)
plt.legend()
plt.xlim(0,1)
plt.ylim(0,1)
file_path = (r"/home/hpc/b129dc/b129dc22/Analysis/stereo_net_roc.png")
plt.savefig(file_path)
plt.show()

['203195171', '2526222', '39126115', '9227178', '14913244', '20413093', '25524679', '40204289', '94285256', '150133253', '20558117', '256201269', '416282', '96173210', '15171159', '206132181', '257147137', '42137170', '9716334', '1526190', '20715796', '258216262', '4279242', '974126', '1537527', '2085035', '2594722', '43229198', '9833127', '154213123', '20935163', '26016181', '44121277', '99196136', '1545216', '21025892', '261227119', '45281252']
['24590248', '323173', '8312234', '14119219', '19425536', '246120151', '323440', '851976', '14240153', '1952918', '247289268', '33139299', '8687293', '14319160', '19622226', '24822278', '34221240', '875292', '144165232', '198113108', '249115221', '3523564', '88257180', '145151168', '199284212', '25094225', '3697109', '8985202', '146108141', '200251215', '251118296', '37112267', '9044102', '147203295', '20145130', '252205199', '38995', '9178162', '1486646']
182 files will be used for training
39 files will be used for validating
38 files will b

NameError: name 'current_file' is not defined

In [3]:
!pip install keras_tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00
